In [ ]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import dash_leaflet as dl
import geopandas
import plotly.graph_objects as go
from wijklabels.report import COLORS

neighborhoods = geopandas.read_file("labels_neighborhood_geom_4326.fgb").set_index("buurtnaam")
nbhd_long = neighborhoods.drop(columns=["geometry", "buurtcode"]).melt(var_name="Energielabel", value_name="Percentage", ignore_index=False)
nbhd_long["Percentage"] = nbhd_long["Percentage"] * 100 

def plot_buurt(buurtnaam):
    fig = go.Figure(data=[go.Bar(
        x=nbhd_long.loc[buurtnaam].Energielabel,
        y=nbhd_long.loc[buurtnaam].Percentage,
        marker_color=list(COLORS.keys()),
        hovertemplate = "%{x}: %{y:.1f}%"
    )])
    fig.update_yaxes(range=[0, 50])
    fig.update_layout(title_text=buurtnaam, xaxis_title="Energielabel", yaxis_title="Percentage (%)")
    return fig

@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-dropdown', component_property='value')
)
def update_graph(buurtnaam):
    fig = plot_buurt(buurtnaam)
    return fig

def get_rect(center: list[float], delta: float = 2):
    return dict(minX=center[1] - delta, maxX=center[1] + delta, minY=center[0] - delta, maxY=center[0] + delta)

#initial_center=[82342.0, 455161.0]
initial_center=[52.08, 4.33]
#initial_center=[480905, 6813218]

basemap_url="https://service.pdok.nl/brt/achtergrondkaart/wmts/v2_0?&service=wmts&request=GetTile&version=1.0.0&dpiMode=7&format=image/png&layer=grijs&styles=default&tileMatrixSet=EPSG:28992&tilematrix={z}&tilecol={x}&tilerow={y}"

app = Dash(__name__)

gj = neighborhoods.to_json()

app.layout = html.Div([
    html.Div(children='Energielabel buurten'),
    dl.Map(children=[
                dl.TileLayer(),
                dl.GeoJSON(
                    data=gj
                )
           ],
           center=initial_center,
           zoom=14,
           style={'height': '50vh'}, id="map"),
    dcc.Dropdown(
        id="controls-and-dropdown",
        options=neighborhoods.index,
        value="Bezuidenhout-Midden",
        clearable=False
    ),
    dcc.Graph(figure={}, id="controls-and-graph"),
])

if __name__ == '__main__':
    app.run(debug=False)